# Проект 5. "Компьютер говорит нет". Credit Scoring.

# Содержание работы:
### 1. [Импорт библиотек](#introduction)
### 2. [Имортируем датасеты](#paragraph1)
### 3. [Просмотр данных](#paragraph2)
### 4. [EDA](#paragraph3)
### 5. [Feature engineering](#paragraph4)
### 6. [Подготовка данных для обучения](#paragraph5)
### 7. [Модель 1. LogisticRegression](#paragraph6)
### 8. [Проверка сбалансированности данных](#paragraph7)
### 9. [Модель 2. LogisticRegressionCV](#paragraph8)
### 10. [Модель 3. LogisticRegression с применением GridSearchCV](#paragraph9)
### 11. [Submission](#paragraph10)





# 1. Импорт библиотек  <a name="introduction"></a>

In [379]:
import numpy as np 
import pandas as pd 
from pandas import Series
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler

import imblearn
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import FitFailedWarning

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('always', FitFailedWarning)

In [380]:
# Зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42

In [381]:
# Зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [382]:
def IQR(x):
    '''Функция для расчета межквартильного размаха'''
    return x.quantile(0.75) - x.quantile(0.25)

def perc25(x):
    '''Функция для расчета нижнего квартиля'''
    return x.quantile(0.25)

def perc75(x):
    '''Функция для расчета верхнего квартиля'''
    return x.quantile(0.75)

def outlier_low(x):
    '''Функция для расчета границы нижнего выброса'''
    return perc25(x) - 1.5*IQR(x)

def outlier_high(x):
    '''Функция для расчета границы верхнего выброса'''
    return perc75(x) + 1.5*IQR(x)

def outlier_diag(x):
    '''
    Функция показывает границы выбросов вычисленные по формулам и гистограму выбросов
    '''
    print(' 25-й перцентиль: {},\n'.format(perc25(x)),
          '75-й перцентиль: {},\n'.format(perc75(x)),
          "IQR: {}, ".format(IQR(x)),
          "Границы выбросов: [{f}, {l}],\n".format(f=outlier_low(x),
                                                 l=outlier_high(x)),
          'Количество выбросов снизу:{},\n'.format(len(data.loc[x<=outlier_low(x)])),
          'Процент выбросов снизу:{}%,\n'.format(len(data.loc[x<=outlier_low(x)])/len(data)*100),
          'Количество выбросов сверху:{},\n'.format(len(data.loc[x>=outlier_high(x)])),
          'Процент выбросов сверху:{}%.'.format(len(data.loc[x>=outlier_high(x)])/len(data)*100))
    
def sns_distplot_boxplot(x):
    '''
    Функция формирует диаграмму распределения данных и диаграмму boxplot выбросов
    '''
    plt.figure(figsize=(18, 10))
    plt.subplot(121)
    sns.distplot(data[x], bins=50, kde = False)
    plt.title(f'{x} distribution\n', fontsize=15)
    plt.xlabel(f'{x}')
    plt.ylabel('quantity (frequency)')
    plt.subplot(122)
    sns.boxplot(data[x])
    plt.title(f'{x} distribution\n', fontsize=15)
    plt.xlabel(f'{x}')
    
def plot_cross_validate(model, X_train, X_test, y_train, y_test, cv=15):
    '''Проверка переобучения модели'''
    
    train_score = cross_val_score(model,
                                  X_train,
                                  y_train,
                                  cv=cv,
                                  scoring='f1',
                                  verbose=False)
    test_score = cross_val_score(model,
                                 X_test,
                                 y_test,
                                 cv=cv,
                                 scoring='f1',
                                 verbose=False)

    avg_f1_train, std_f1_train = train_score.mean(), train_score.std()
    avg_f1_test, std_f1_test = test_score.mean(), test_score.std()
    plt.figure(figsize=(10, 4))
    plt.plot(
        train_score,
        label=f'[Train] F1-score: {avg_f1_train:.2f} $\pm$ {std_f1_train:.2f}',
        marker='.')
    plt.plot(
        test_score,
        label=f'[Test] F1-score: {avg_f1_test:.2f} $\pm$ {std_f1_test:.2f}',
        marker='.')
    plt.ylim([0.02, 1.])
    plt.xlabel('CV iteration', fontsize=15)
    plt.ylabel('F1-score', fontsize=15)
    plt.legend(fontsize=15)

# 2. Имортируем датасеты <a name="paragraph1"></a>

In [383]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
df_train = pd.read_csv(DATA_DIR +'train.csv')
df_test = pd.read_csv(DATA_DIR +'test.csv')
sample_submission = pd.read_csv(DATA_DIR +'sample_submission.csv')

## Описания полей
* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заемщика
* age - возраст заемщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия “хорошей” работы
* bki_request_cnt - количество запросов в БКИ
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заемщика
* foreign_passport - наличие загранпаспорта
* sna - связь заемщика с клиентами банка
* first_time - давность наличия информации о заемщике
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона
* app_date - дата подачи заявки
* default - флаг дефолта по кредиту

# 3. Просмотр данных <a name="paragraph2"></a>

In [384]:
# Посмотрим информацию по датасетам
print('Инфориация по тренировочному датасете:')
df_train.info()
print('-' *40)
print('Инфориация по тестовому датасете:')
df_test.info()

In [385]:
# Посмотрим количество пропусков в датасетах
print('Количество пропусков в тренировочном датасете:')
display(df_train.isnull().sum())
print('-' *40)
print('Количество пропусков в тестовом датасете:')
display(df_test.isnull().sum())

In [386]:
sample_submission.head(5)

In [387]:
df_train.head(5)

In [388]:
# Для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения default, мы его должны предсказать, по этому пока просто заполняем нулями
data = pd.concat([df_train, df_test], ignore_index=True)
data.head(5)

In [389]:
# Посмотрим целевую переменную на тренировочном датасете, т.к. на тестовом у нас все равно 0
print('Количество не дефолтов:')
display(df_train.loc[df_train['default']==0, 'default'].count())
print('-'*40)
print('Количество дефолтов:')
display(df_train.loc[df_train['default']==1, 'default'].count())
print('-'*40)
data.default.value_counts(ascending=True).plot(kind='barh')

### Выводы по данным:

1. Всего исходных признаков: 18 (не учитывая client_id, т.к. она не несет смысловой нагрузки и целевую переменную)
2. Количество записей в тренировочном датасете: 73799
3. Количество записей в тестовом датасете: 36349
4. Бинарные переменные: sex, car, car_type, good_work, foreign_passport
5. Категориальные переменные: education, region_rating, home_address, work_address, sna, first_time
6. Числовые переменные: age, decline_app_cnt, score_bki, bki_request_cnt, income
7. Временная переменная: app_date
8. Целевая переменная: default
9. Пропуски: В датасете пропуски имеет только признак education: 307 в тренировочном и 171 в тестовом

In [390]:
# Разделим сразу признаки по категориям
num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt'] 
cat_cols = ['education','region_rating', 'work_address','home_address','sna','first_time'] 
bin_cols = ['sex','car','car_type','good_work','foreign_passport']
date_cols = ['app_date']

# 4. EDA <a name="paragraph3"></a>

### 4.1 Анализ данных в колонке 'education' <a name="#subparagraph1"></a>

In [391]:
# Посмотрим на сначала на столбец education с пропусками 
print('Распределение заполенных значений признака:')
display(data.education.value_counts(ascending=True))
print('-' *40)
print('Количество пропусков:')
display(data.education.isnull().sum())
print('-' *40)
print('Количество пропусков в %:') 
display(round((data.education.isnull().sum()/len(data))*100,2))
print('-' *40)
plt.figure(figsize=(15,5))
data.education.value_counts(ascending=True).plot(kind='barh')

In [392]:
# Проверим гиппотезу, что данный признак зависит от других, и может получится выявить интуитивно понятный способ заполнения
# Преобразуем значение признака в уникальное число в копии датасета и удалим пропуски
data_edu = data.dropna().copy()
label_encoder = LabelEncoder()
data_edu['education'] = label_encoder.fit_transform(data_edu['education'])
print(dict(enumerate(label_encoder.classes_)))
data_edu.head(2)

In [393]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.corr(), vmin=0, vmax=1, annot = True)

Больше всего образование зависит от признаков income и good_work

In [394]:
#print('Зависимость образования от наличия хорошей работы:')
display(data.groupby(['education', 'good_work'])['good_work'].count())
print('-' *40)
print('Зависимость образования от зарплаты:')
display(data.groupby('education')['income'].agg(['min', 'median', 'max']))
print('-' *40)
plt.figure(figsize=(10, 8))
plt.suptitle("Зависимость образования от зарплаты")
sns.boxplot(x="education", y="income", data=data, showfliers=False)

Пропуски можно было бы заполнить несколькими способами:
* удалить пропуски, т.к. их отношение к общей выборке не очень велико
* построить отдельную модель для предсказания данного признака
* отдельно проведено исследование, что данный признак больше всего зависит от признаков income и good_work. Но интуитивно не понятно какое именно значение проставлять в пропущенных значениях.
* заполнить наиболее часто встречающимся значеним

Количество пропусков в данном признаке = 0,4%. Это не большое значение, для постоения модели выберем способ заполнения наиболее часто встречающимся значеним.

In [395]:
# Посмотрим моду признака education
print('Мода признака education:')
display(data['education'].mode()[0])

In [396]:
# Заполним пропуски наиболее часто встречающимся значеним
data['education'] = data['education'].fillna('SCH')
print('Количество пропусков:')
display(data.education.isnull().sum())

In [397]:
# Пребразуем категориальные значения в целочисленные значения
education_dict = {'SCH': 1,
                  'GRD': 2,
                  'UGR': 3,
                  'PGR': 4,
                  'ACD': 5}

data['education'] = data['education'].map(education_dict)

### 4.2 Анализ данных в колонке 'age' <a name="#subparagraph2"></a>

In [398]:
# Посмотрим распределение и выбросы на графиках
sns_distplot_boxplot('age')

In [399]:
# Проверим выбросы по формуле
outlier_diag(data['age'])

Данный признак имеет тяжёлый правый хвост, сделаем распределение данных более нормальным, прологорифмировав их. Выбросов нет.

In [400]:
# Прологорифмируем данные 
#data['age'] = np.log(data['age'] + 1)

In [401]:
# Посмотрим на распределение после нормализации данных
#sns_distplot_boxplot('age')

In [402]:
# Проверим выбросы по формуле
#outlier_diag(data['age'])

Распределение стало более нормальным.

### 4.3 Анализ данных в колонке 'decline_app_cnt' <a name="#subparagraph3"></a>

In [403]:
# Проверим выбросы по формуле
sns_distplot_boxplot('decline_app_cnt')

In [404]:
# Посмотрим количество значений
data.decline_app_cnt.value_counts()

In [405]:
# Проверим выбросы по формуле
outlier_diag(data['decline_app_cnt'])

In [406]:
# Попробуем сделать данный признак категориальным Основные значения распределены от 0 до 3.
# Все значения выше 3 приравняем к 3
data['decline_app_cnt'] = data['decline_app_cnt'].apply(lambda x: x if x < 3 else 3)

In [407]:
# Обновим списоки числовых и категориальных переменных
num_cols.remove("decline_app_cnt")
cat_cols.append('decline_app_cnt')

In [408]:
data['decline_app_cnt'].unique()

In [409]:
# Посмотрим сколько значений decline_app_cnt > 0 в положительном default
#data.loc[(data['default']==1) & (data['decline_app_cnt']>3),'decline_app_cnt'].count()

Данный признак имеет тяжёлый правый хвост, сделаем распределение данных более нормальным, прологорифмировав их. 
Основное распределение признака лежит в 0, поэтому все значения кроме 0 считаются выбросоми. Выбросы удалять не будем, т.к. у нас и так не большая выборка положительного дефолта, уменьшать ее на 2667 значений, может плохо сказаться на обучении.  

In [410]:
# Прологорифмируем данные и посмотрим на распределение после нормализации данных
#data['decline_app_cnt'] = np.log(data['decline_app_cnt'] + 1)
#sns_distplot_boxplot('decline_app_cnt')

### 4.4 Анализ данных в колонке 'score_bki' <a name="#subparagraph4"></a>

In [411]:
# Посмотрим распределение и выбросы на графиках
sns_distplot_boxplot('score_bki')

In [412]:
# Проверим выбросы по формуле после логарифмирования
outlier_diag(data['score_bki'])

Распределение у данного признака выглядит нормальным + логорифм от отрицательных чисел брать нельзя. Выбросы удалять не будем, их не значительное количество. Возможно вернемся к ним потом. 

### 4.5 Анализ данных в колонке 'income' <a name="#subparagraph5"></a>

In [413]:
# Посмотрим распределение и выбросы на графиках
sns_distplot_boxplot('income')

In [414]:
# Проверим выбросы по формуле
outlier_diag(data['income'])

In [415]:
# Посмотрим сколько значений income > 90000 в положительном default
data.loc[(data['default']==1) & (data['income']>90000),'income'].count()

Данный признак имеет тяжёлый правый хвост, сделаем распределение данных более нормальным, прологорифмировав их. 
Выбросы у данного признака только сверху. Выбросы удалять пока не будем, имеет место быть высокая заработная плата и мы отсечем 7749 значений, что может плохо сказаться на дальнейших предсказаниях модели.  

In [416]:
# Прологорифмируем данные и посмотрим на распределение после нормализации данных
data['income'] = np.log(data['income'] + 1)
sns_distplot_boxplot('income')

In [417]:
# Проверим выбросы по формуле после логарифмирования
outlier_diag(data['income'])

Распределение стало более нормальным, уменьшилось количество выбросов.

### 4.6 Анализ данных в колонке 'bki_request_cnt' <a name="#subparagraph6"></a>

In [418]:
# Посмотрим распределение и выбросы на графиках
sns_distplot_boxplot('bki_request_cnt')

In [419]:
# Проверим выбросы по формуле
outlier_diag(data['bki_request_cnt'])

In [420]:
# Посмотрим сколько значений income > 90000 в положительном default
data.loc[(data['default']==1) & (data['bki_request_cnt']>7.5),'bki_request_cnt'].count()

Данный признак имеет тяжёлый правый хвост, сделаем распределение данных более нормальным, прологорифмировав их. 
Выбросы уданного признака тоже только сверху. Выбросы удалять пока не будем, их не так много в целом и не много в положительном дефолте.  

In [421]:
# Прологорифмируем данные и посмотрим на распределение после нормализации данных
data['bki_request_cnt'] = np.log(data['bki_request_cnt'] + 1)
sns_distplot_boxplot('bki_request_cnt')

In [422]:
# Проверим выбросы по формуле после логарифмирования
outlier_diag(data['bki_request_cnt'])

Распределение стало более нормальным, уменьшилось количество выбросов.

### 4.7 Анализ данных в колонке 'app_date' <a name="#subparagraph7"></a>

In [423]:
# Посмотрим текущий формат даты
display(data['app_date'].head(5))
print('-'*40)
print('Формат первой строки:', type(data['app_date'][0]))

In [424]:
# Преобразуем признак app_date в формат datetime
data['app_date'] = pd.to_datetime(data['app_date'], format='%d%b%Y')
display(data['app_date'].head(5))

In [425]:
# Посмотрим период данных, которые представлены в датасете
print('Начальная дата:')
display(data['app_date'].min())
print('Конечная дата:')
display(data['app_date'].max())

Период данных в датасете представлен за 4 месяца, период с 1 января 2014 года по 30 апреля 2014 года.
Попробуем преабразовать данный признак в учет дней с подачи первой заявки, тем самым преобразуем его в числовое значение и проверим корреляцию.

In [426]:
# Преобразуем признак в учет дней с подачи первой заявки
data['app_date'] = data['app_date'].apply(lambda x: (x - data['app_date'].min()).days)
data['app_date'].head(5)

In [427]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.corr(), vmin=0, vmax=1, annot = True)

Из таблицы сильная корреляция с признаком client_id

In [428]:
# Посмотрим на графике зависимость client_id от учета дней с подачи первой заявки
sns.scatterplot(x='client_id',y='app_date',data=data)

По данному графику и из таблицы корреляции видно, что в датасете прямая зависимость присвоения id от дня подачи заявки. 

### 4.8 Проверка числовых переменных <a name="#subparagraph8"></a>

In [429]:
# Добавим признак числовые переменные
num_cols.append('app_date')

In [430]:
# Посмотрим числовые переменные на графиках Boxplots для тренировочной выборки, т.к. в тестовой целевая переменная равна 0
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='default', y=column, data=data[data['sample']==1],ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [431]:
for col in num_cols:
    get_boxplot(col)

Выводы:

* Клиенты с дефолтом младше
* Клиенты с имеют больше отмененных заявок
* Клиенты с дефолтом имеют более высокий балл БКИ
* Клиенты с дефолтом имеют более низкую зарплату
* Клиенты с дефолтом имеют больше запросов в БКИ
* Клиенты с дефолтом обращались за кредитом ближе к началу подачи всех заявок, ближе к апрелю кредитов одабривалось больше

### 4.9 Проверка категориальных переменных <a name="#subparagraph9"></a>

In [432]:
# Посмотрим категориальные переменные на графиках Boxplots для тренировочной выборки, т.к. в тестовой целевая переменная равна 0
def get_boxplot(column):
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='default', y=column, data=data[data['sample']==1],ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [433]:
for col in cat_cols:
    get_boxplot(col)

Выводы:
* Люди имеют примерно одинаковый уровень образования в двух категориях дефолта
* Чем ниже рейтинг региона тем вероятнее дефолт
* Дефолт вероятнее у людей, которые берут кредит первый раз 
* Дефолт вероятнее у людей, у которых связь с клиентами банка выше.

# 5. Feature engineering <a name="paragraph4"></a>

In [434]:
# Посмотрим на матрицу корреляций
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.corr(), vmin=0, vmax=1, annot = True)

* На целевую переменную больше всего влияют score_bki, decline_app_cnt, sna
* Сильно скоррелированы client_id и app_date. client_id удалим далее, т.к. он не несет смысловой нагрузки
* Сильно скоррелированы home_adress и work_adress. Посмотрим далее, может удалим один из признаков, с наименьшей значимостью для обучения модели.

In [435]:
# Посмотрим влияние численных признаков на целевую переменную
plt.figure(figsize=(8, 8))
imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [436]:
# Для бинарных признаков используем LabelEncoder для преобразрвания в целочисленные значения
label_encoder = LabelEncoder()

for column in bin_cols:
    print(column)
    data[column] = label_encoder.fit_transform(data[column])
    print(dict(enumerate(label_encoder.classes_))) 

In [437]:
# Посмотрим распределение признака income
for x in (data['income'].value_counts()).head(10).index:
    data['score_bki'][data['income'] == x].hist(bins=100)
plt.show()

In [438]:
# Найдем среднее значение из сгруппированных 
mean_score_bki_income_gr = data.groupby(['income'])['score_bki'].mean()
data['mean_score_bki_income'] = data['income'].apply(lambda x: mean_score_bki_income_gr[x])

In [439]:
# Центрируем score_bki по сгруппированному по зарплате его же значения
data['score_bki_income_cent'] = data['score_bki'] - data['mean_score_bki_income']
data.sample(2)

In [440]:
# Посмотрим распределение признака region_rating
for x in (data['region_rating'].value_counts()).head(10).index:
    data['score_bki'][data['region_rating'] == x].hist(bins=100)
plt.show()

In [441]:
# Найдем среднее значение из сгруппированных 
mean_score_bki_rr_gr = data.groupby(['region_rating'])['score_bki'].mean()
data['mean_score_bki_rr'] = data['region_rating'].apply(lambda x: mean_score_bki_rr_gr[x])

In [442]:
# Центрируем "score_bki_rr_cent" по сгруппированному по рейтингу региона его же значения
data['score_bki_rr_cent'] = data['score_bki'] - data['mean_score_bki_rr']
data.sample(2)

In [443]:
# Нормируем score_bki по возрасту
data['norm_score_bki_age'] = data['score_bki_rr_cent'] / data['age']

In [444]:
# Нормируем зарплату по возрасту
data['norm_income_age'] = data['income']*data['age'] 

In [445]:
# Обновим список с численными переменными
new_features = ['score_bki_income_cent', 'score_bki_rr_cent', 'norm_score_bki_age', 'norm_income_age']
for i in new_features:
    num_cols.append(i)

In [446]:
# Посмотрим влияние категориальных и бинарных признаков на целевую переменную
plt.figure(figsize=(8, 8))
imp_cat = Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [447]:
# !Ухудшило модель # Попробуем удалить work_adress, т.к. как он сильно скоррелирован с home_adress и имеет меньшее влиянее на целевую переменную
#data = data.drop(['work_address'], axis=1)
#cat_cols.remove('work_address')

In [448]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data[bin_cols].corr(), vmin=0, vmax=1, annot = True)

In [449]:
# Столбцы car и car_type сильно скоррелированы, объединим данные столбцы
data['car_type'] = data['car']+data['car_type']
data = data.drop(['car'], axis=1)
bin_cols.remove('car')

In [450]:
# Стандартизируем числовые переменные
data[num_cols] = pd.DataFrame(StandardScaler().fit_transform(data[num_cols]), columns = data[num_cols].columns)

In [451]:
# Создадим отдельную бинарную переменную для каждого категориального признака
data = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols) # categorical dummies

In [452]:
# Выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample','client_id'], axis=1)
test_data = data.query('sample == 0').drop(['sample','client_id'], axis=1)

In [453]:
# Посмотрим еще раз на влияние признаков на целевую переменную
columns_analisys = train_data.columns.to_list()
columns_analisys.remove('default')
plt.figure(figsize=(8, 8))
imp_num = pd.Series(f_classif(train_data.drop(['default'], axis=1), train_data['default'])[0], index = columns_analisys)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Выводы:
* Самое большое влияние на целевую переменную оказывают признаки: score_bki, decline_app_cnt, sna_1, sna_4
* Нмкакого влияния не оказывают признаки: region_rating_60, home_address_3, education_3. Удалим их.

In [454]:
# Удалим столбцы, которые меньше всего влияют на целевую переменную
train_data = train_data.drop(['region_rating_60', 'home_address_3', 'education_3', 'mean_score_bki_income', 'mean_score_bki_rr'], axis=1)
test_data = test_data.drop(['region_rating_60', 'home_address_3', 'education_3', 'mean_score_bki_income', 'mean_score_bki_rr'], axis=1)

# Удалим столбцы, которые меньше всего влияют на целевую переменную
#train_data = train_data.drop(['region_rating_60', 'home_address_3', 
#                              'education_3', 'work_address_3', 
#                              'work_address_2', 'mean_score_bki_income', 
#                              'mean_score_bki_rr'], axis=1)
#test_data = test_data.drop(['region_rating_60', 'home_address_3', 
#                            'education_3', 'work_address_3', 
#                            'work_address_2', 'mean_score_bki_income', 
#                            'mean_score_bki_rr'], axis=1)

# 6. Подготовка данных для обучения <a name="paragraph5"></a>

In [455]:
# Формируем данные для обучения
X = train_data.drop(['default'], axis=1)
y = train_data['default'].values # наш таргет

In [456]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [457]:
# Проверяем разбивку
test_data.shape, train_data.shape, X.shape, X_train.shape, X_valid.shape

# 7. Модель 1. LogisticRegression. <a name="paragraph6"></a>

In [458]:
# Обучаем первую модель с помощью LogisticRegression
model = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=RANDOM_SEED)
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

In [459]:
# Посмотрим график метрики ROC-AUC
probs = model.predict_proba(X_valid)
probs = probs[:,1]

fpr, tpr, threshold = roc_curve(y_valid, probs)
roc_auc = roc_auc_score(y_valid, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [460]:
# Посмотрим confusion_matrix
cm = confusion_matrix(y_valid, y_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [461]:
# Посмотрим значения precision и recall
precision = precision_score(y_valid, y_pred)
recall = recall_score(y_valid, y_pred)
print('precision: {:.2f}\nrecall: {:.2f}'.format(precision, recall))

In [462]:
# Посмотрим значение F1
f1 = f1_score(y_valid, y_pred)
print('F1-score: {:.2f}'.format(f1))

In [463]:
# Проверим модель на переобучение
plot_cross_validate(model,
                    X_train, 
                    X_valid, 
                    y_train, 
                    y_valid,
                    cv=5)

Целевая метрика ROC-AUC достаточно высокая, но полученные значения метрик precision и recall говорят, что алгоритм мало назначает объектам положительный класс. Чем выше значение F1-score, тем лучше. Как видим, наш алгоритм работает неудовлетворительно.

# 8. Проверка сбалансированности данных <a name="paragraph7"></a>

## 8.1 Under-sampling

In [464]:
train_data_balance = train_data.copy()
# Посмотрим на распределение целевой переменной
count_class_0, count_class_1 = train_data_balance.default.value_counts()
print('Количество значений default = 1:', count_class_1,'\nКоличество значений default = 0:', count_class_0)

In [465]:
# Создадим новый датасет с разбивкой целевой переменной
train_data_balance_class_0 = train_data_balance[train_data_balance['default'] == 0]
train_data_balance_class_1 = train_data_balance[train_data_balance['default'] == 1]

In [466]:
# Сбалансируем выборку целевой переменной under-sampling
train_data_balance_class_0_under = train_data_balance_class_0.sample(count_class_1)
train_data_balance = pd.concat([train_data_balance_class_0_under, train_data_balance_class_1], axis=0)

print('Random under-sampling:')
print(train_data_balance.default.value_counts())

train_data_balance.default.value_counts().plot(kind='bar', title='Count (target)');

In [467]:
# Формируем данные для обучения + under-sampling
X_1 = train_data_balance.drop(['default'], axis=1)
y_1 = train_data_balance['default'].values # наш таргет

In [468]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_1_train, X_1_valid, y_1_train, y_1_valid = train_test_split(X_1, y_1, test_size=0.2, random_state=RANDOM_SEED)

In [469]:
# Обучаем модель с помощью LogisticRegression + under-sampling
model_1 = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=RANDOM_SEED)
model_1.fit(X_1_train, y_1_train)
y_1_pred = model_1.predict(X_1_valid)

In [470]:
# Посмотрим график метрики ROC-AUC + under-sampling
probs_1 = model_1.predict_proba(X_1_valid)
probs_1 = probs_1[:,1]

fpr, tpr, threshold = roc_curve(y_1_valid, probs_1)
roc_auc = roc_auc_score(y_1_valid, probs_1)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [471]:
# Посмотрим confusion_matrix + under-sampling
cm = confusion_matrix(y_1_valid, y_1_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [472]:
# Посмотрим значения precision и recall + under-sampling
precision = precision_score(y_1_valid, y_1_pred)
recall = recall_score(y_1_valid, y_1_pred)
print('precision: {:.2f}\nrecall: {:.2f}'.format(precision, recall))

In [473]:
# Посмотрим значение F1 + under-sampling
f1_1 = f1_score(y_1_valid, y_1_pred)
print('F1-score: {:.2f}'.format(f1_1))

In [474]:
# Проверим модель на переобучение
plot_cross_validate(model_1,
                    X_1_train, 
                    X_1_valid, 
                    y_1_train, 
                    y_1_valid,
                    cv=5)

Целевая метрика ROC-AUC  высокая, полученные значения метрик precision и recall выросли в 2 раза по сравнению с первой моделью, качество предсказаний . Значение F1-score, так же выросло в 2 раза. Как видим, наш алгоритм стал работать более удовлетворительно. Но и пармаетров non-default стало меньше в 6 раз.

## 8.2 Over-sampling

In [475]:
# Сбалансируем выборку целевой переменной over-sampling
oversampling = int(count_class_0/count_class_1)
for i in range(oversampling):
    train_data_oversampling = train_data.append(train_data_balance_class_1).reset_index(drop=True)

In [476]:
# Пример кода с RandomOverSampler, результат получается хуже, не хватило времени попробовать подобрать sampling_strategy
#X_over = train_data.drop(['default'], axis=1)
#y_over = train_data['default'].values # наш таргет
#oversample = RandomOverSampler(sampling_strategy=0.5)
#X_over1, y_over1 = oversample.fit_resample(X_over, y_over)
#print('Resampled dataset shape %s' % Counter(y_over1))

In [477]:
# Формируем данные для обучения + over-sampling
X_1 = train_data_oversampling.drop(['default'], axis=1)
y_1 = train_data_oversampling['default'].values # наш таргет

In [478]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_1_train, X_1_valid, y_1_train, y_1_valid = train_test_split(X_1, y_1, test_size=0.2, random_state=RANDOM_SEED)

In [479]:
# Обучаем модель с помощью LogisticRegression + over-sampling
model_1 = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=RANDOM_SEED)
model_1.fit(X_1_train, y_1_train)
y_1_pred = model_1.predict(X_1_valid)

In [480]:
# Посмотрим график метрики ROC-AUC + over-sampling
probs_1 = model_1.predict_proba(X_1_valid)
probs_1 = probs_1[:,1]

fpr, tpr, threshold = roc_curve(y_1_valid, probs_1)
roc_auc = roc_auc_score(y_1_valid, probs_1)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [481]:
# Посмотрим confusion_matrix + over-sampling
cm = confusion_matrix(y_1_valid, y_1_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [482]:
# Посмотрим значения precision и recall + over-sampling
precision = precision_score(y_1_valid, y_1_pred)
recall = recall_score(y_1_valid, y_1_pred)
print('precision: {:.2f}\nrecall: {:.2f}'.format(precision, recall))

In [483]:
# Посмотрим значение F1 + over-sampling
f1_1 = f1_score(y_1_valid, y_1_pred)
print('F1-score: {:.2f}'.format(f1_1))

In [484]:
# Проверим модель на переобучение
plot_cross_validate(model_1,
                    X_1_train, 
                    X_1_valid, 
                    y_1_train, 
                    y_1_valid,
                    cv=5)

Целевая метрика ROC-AUC  высокая, полученные значения метрик precision и recall выросли по сравнению с первой моделью1, качество предсказаний . Значение F1-score, так же выросло. Как видим, наш алгоритм стал работать более удовлетворительно.

# 9. Модель 2. LogisticRegressionCV <a name="paragraph8"></a>

In [485]:
# Формируем данные для обучения
X_lrcv = train_data.drop(['default'], axis=1)
y_lrcv = train_data['default'].values # наш таргет

In [486]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_lrcv_train, X_lrcv_valid, y_lrcv_train, y_lrcv_valid = train_test_split(X_lrcv, y_lrcv, test_size=0.2, random_state=RANDOM_SEED)

In [487]:
# Проверяем распределение
test_data.shape, train_data.shape, X_lrcv.shape, X_lrcv_train.shape, X_lrcv_valid.shape

In [488]:
# Добавим типы регуляризации
penalty = ['l1', 'l2', 'elasticnet']
# Обучаем модель с помощью LogisticRegressionCV
model_lrcv = LogisticRegressionCV(solver='saga', l1_ratios=[0, 1], Cs=np.logspace(0, 4, 10), 
                                  penalty='elasticnet', cv=5, random_state=RANDOM_SEED, 
                                  class_weight='balanced', scoring='roc_auc')
model_lrcv.fit(X_lrcv_train, y_lrcv_train)
y_pred_lrcv = model_lrcv.predict(X_lrcv_valid)

In [489]:
# Посмотрим график метрики ROC-AUC
probs_lrcv = model_lrcv.predict_proba(X_lrcv_valid)
probs_lrcv = probs_lrcv[:,1]

fpr, tpr, threshold = roc_curve(y_lrcv_valid, probs_lrcv)
roc_auc = roc_auc_score(y_lrcv_valid, probs_lrcv)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [490]:
# Посмотрим confusion_matrix
cm = confusion_matrix(y_lrcv_valid, y_pred_lrcv)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [491]:
# Посмотрим значения precision и recall
precision = precision_score(y_lrcv_valid, y_pred_lrcv)
recall = recall_score(y_lrcv_valid, y_pred_lrcv)
print('precision: {:.2f}\nrecall: {:.2f}'.format(precision, recall))

In [492]:
# Посмотрим значение F1
f1_1 = f1_score(y_lrcv_valid, y_pred_lrcv)
print('F1-score: {:.2f}'.format(f1_1))

По сравнению с первой моделью качество не выросло.

In [493]:
# Проверим модель на переобучение
plot_cross_validate(model_lrcv,
                    X_lrcv_train, 
                    X_lrcv_valid, 
                    y_lrcv_train, 
                    y_lrcv_valid,
                    cv=5)

# 10. Модель 3. LogisticRegression с применением GridSearchCV <a name="paragraph9"></a>

In [494]:
# Формируем данные для обучения. Из 3 наборов данных получается самая худшая моель.
#X_3 = train_data.drop(['default'], axis=1)
#y_3 = train_data['default'].values # наш таргет

In [495]:
# Формируем данные для обучения + under-sampling. Модель хуже чем с over-sampling
#X_3 = train_data_balance.drop(['default'], axis=1)
#y_3 = train_data_balance['default'].values # наш таргет

In [496]:
# Формируем данные для обучения + over-sampling
X_3 = train_data_oversampling.drop(['default'], axis=1)
y_3 = train_data_oversampling['default'].values # наш таргет

In [497]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_3_train, X_3_valid, y_3_train, y_3_valid = train_test_split(X_3, y_3, test_size=0.2, random_state=RANDOM_SEED)

In [498]:
# Проверяем распределение
test_data.shape, train_data.shape, X_3.shape, X_3_train.shape, X_3_valid.shape

In [499]:
# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)
iter_ = 1000
# Создадим гиперпараметры
hyperparameters = [
    {'penalty': ['l1'], 
     'C': C,
     'solver': ['liblinear', 'saga'], 
     'class_weight':['balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_]},
    {'penalty': ['l2'], 
     'C': C,
     'solver': ['newton-cg', 'liblinear', 'sag', 'saga'], 
     'class_weight':['balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_]},
    {'penalty': ['none'], 
     #'C': C,
     'solver': ['newton-cg', 'sag', 'saga'], 
     'class_weight':['balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_]},
]

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
# указываем модель (в нашем случае лог регрессия), гиперпараметры
#modelcv = LogisticRegression(solver = 'sag', penalty='l2', max_iter=1000, class_weight='balanced', random_state=RANDOM_SEED)
modelcv = LogisticRegression(penalty='l2', max_iter=1000, class_weight='balanced', random_state=RANDOM_SEED)
#modelcv.fit(X_3_train, y_3_train)

# Обучаем модель
gridsearch = GridSearchCV(modelcv, hyperparameters, scoring='f1', n_jobs=-1, cv=5)
model_3 = gridsearch.fit(X_3_train, y_3_train)

In [500]:
# Печатаем параметры
best_parameters = model_3.best_estimator_.get_params()
print(f'Лучшие значения параметров:') 
for param_name in sorted(best_parameters.keys()):
        print(f'  {param_name} => {best_parameters[param_name]}')

In [501]:
model_3.best_estimator_

In [502]:
# Применим лучшие гиперпараметры
#model_3 = gridsearch.best_estimator_
# Предсказываем
probs_3 = model_3.predict_proba(X_3_valid)[:,1]
y_3_pred = model_3.predict(X_3_valid)

In [503]:
# Посмотрим график метрики ROC-AUC
fpr, tpr, threshold = roc_curve(y_3_valid, probs_3)
roc_auc = roc_auc_score(y_3_valid, probs_3)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [504]:
# Посмотрим confusion_matrix
cm = confusion_matrix(y_3_valid, y_3_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [505]:
# Посмотрим значения precision и recall
precision = precision_score(y_3_valid, y_3_pred)
recall = recall_score(y_3_valid, y_3_pred)
print('precision: {:.2f}\nrecall: {:.2f}'.format(precision, recall))

In [506]:
# Посмотрим значение F1
f1_3 = f1_score(y_3_valid, y_3_pred)
print('F1-score: {:.2f}'.format(f1_3))

Лучший показатель предсказания в рамках LogisticRegression получен с помощью Cross-validation, GridSearchCV и over-sampling.

# 11. Submission <a name="paragraph10"></a>

In [508]:
X_test = test_data.drop(['default'], axis=1)
y_pred = model_3.predict_proba(X_test)
submission = pd.DataFrame(data={'client_id':df_test['client_id'], 'default':y_pred[:,1]})
submission.to_csv('submission.csv', index=False)
submission